In [1]:
import sqlite3
import json

# Load JSON
with open("../data/cruise_literature/cruise_literature_parsed.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Connect to SQLite DB 
conn = sqlite3.connect("../data/dataset.db")
cur = conn.cursor()

# Create tables
cur.executescript("""
CREATE TABLE IF NOT EXISTS reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    source_url TEXT
);

CREATE TABLE IF NOT EXISTS authors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review_id INTEGER,
    name TEXT,
    FOREIGN KEY (review_id) REFERENCES reviews(id)
);

CREATE TABLE IF NOT EXISTS abstracts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review_id INTEGER,
    section TEXT,
    content TEXT,
    FOREIGN KEY (review_id) REFERENCES reviews(id)
);

CREATE TABLE IF NOT EXISTS pico (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    review_id INTEGER,
    category TEXT,
    term TEXT,
    FOREIGN KEY (review_id) REFERENCES reviews(id)
);
""")

# Insert data
for review in data:
    title = review.get("title")
    url = review.get("source_url")

    # Insert review and get its ID
    cur.execute("INSERT INTO reviews (title, source_url) VALUES (?, ?)", (title, url))
    review_id = cur.lastrowid

    # Insert authors
    for author in review.get("authors", []):
        cur.execute("INSERT INTO authors (review_id, name) VALUES (?, ?)", (review_id, author))

    # Insert abstract sections
    for section, content in review.get("abstract", {}).items():
        cur.execute("INSERT INTO abstracts (review_id, section, content) VALUES (?, ?, ?)",
                    (review_id, section, content))

# Commit and close
conn.commit()
conn.close()